# torchtext

Pytorch에서는 torchtext라는 자연어 처리 라이브러리를 제공함.
아래와 같은 기능들을 제공한다.

- 파일 로드하기(File Loading) : 다양한 포맷의 코퍼스를 로드합니다.
- 토큰화(Tokenization) : 문장을 단어 단위로 분리해줍니다.
- 단어 집합(Vocab) : 단어 집합을 만듭니다.
- 정수 인코딩(Integer encoding) : 전체 코퍼스의 단어들을 각각의 고유한 정수로 맵핑합니다.
- 단어 벡터(Word Vector) : 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어줍니다. 랜덤값으로 초기화한 값일 수도 있고, 사전 훈련된 임베딩 벡터들을 로드할 수도 있습니다.
- 배치화(Batching) : 훈련 샘플들의 배치를 만들어줍니다. 이 과정에서 패딩 작업(Padding)도 이루어집니다.

In [1]:
!pip install torchtext==0.6.0

# 자꾸 오류나서 이전버전을 설치해줬음.

# 데이터 불러와서 훈련/테스트 분리

In [2]:
import torchtext


import urllib.request
import pandas as pd

In [3]:
# vesion check
torchtext.__version__

In [4]:
# download IMDB review data
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")


df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')     # len(df) -> 50000
df.head()

In [5]:
train_df = df[:25000]
test_df = df[25000:]

In [6]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

# 필드 정의하기(torchtext.data)

- sequential : 시퀀스 데이터 여부. (True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
- is_target : 레이블 데이터 여부. (False가 기본값)
- fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다

In [8]:
from torchtext import data


TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  fix_length=20)


LABEL = data.Field(sequential=False, 
                   use_vocab=False, 
                   batch_first=False, 
                   is_target=True)   # 이게 label이야?

# 데이터 셋 만들기

In [9]:
from torchtext.data import TabularDataset

`TabularDataset`은 데이터를 불러오면서 필드에서 정의했던 방법으로 **토큰화를 수행!**
- path : 파일이 위치한 경로.
- format : 데이터의 포맷.
- fields : 위에서 정의한 필드를 지정. 첫번째 원소는 데이터 셋 내에서 해당 필드를 호칭할 이름, 두번째 원소는 지정할 필드.
- skip_header : 데이터의 첫번째 줄은 무시.

In [12]:
train_data, test_data = TabularDataset.splits(
    path = '.', train = 'train_data.csv', test = 'test_data.csv', format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)], skip_header = True)

In [18]:
# 각 데이터의 개수 확인
print(f"훈련 샘플의 개수: {len(train_data)}")
print(f"테스트 샘플의 개수: {len(test_data)}")

In [24]:
df.loc[0].review

In [19]:
# vars()를 통해서 주어진 인덱스의 샘플을 확인 할 수 있음.
print(vars(train_data[0]))

In [31]:
# 필드 구성 확인
train_data.fields.items()
# 내가 했을 땐 왜 안 됐지...

# 단어 집합 만들기(Vocab)

정수 인코딩 작업이 필요함. 
`build_vocab()`을 사용해 단어 집합을 생성
- min_freq: 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가.
- max_size: 단어 집합의 최대 크기를 지정

In [34]:
# 10번 이상 등장하고, 단어 사전은 최대 10000사이즈로 만들것임!
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [35]:
# unk과 pad를 포함해 총 10002개의 단어사전이 완성되었음!
print(f"단어 집합의 크기: {len(TEXT.vocab)}")

이 집합 내의 단어들은 `.stoi`을 통해 확인 가능함

In [36]:
print(TEXT.vocab.stoi)

# 토치텍스트의 데이터로더 만들기

In [37]:
from torchtext.data import Iterator

In [38]:
# 임의의 배치사이즈 설정
batch_size = 5


train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

In [42]:
print(f"훈련 데이터의 미니 배치 수 : {len(train_loader)}")
print(f"테스트 데이터의 미니 배치 수 : {len(test_loader)}")
# 25000개의 데이터를 5개의 배치로 묶어주었기에 총 배치의 개수는 5000개가 돤다.

In [43]:
# 첫번 째 미니 배치를 받아와보기
batch = next(iter(train_loader))

In [45]:
type(batch)

In [44]:
batch

In [53]:
batch.text
# 여기서 나타나는 0은 [unk] token임

In [54]:
batch.text[0]